In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns

from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
def read_all_data():
    data = pd.read_csv('/kaggle/input/rossmann-store-sales/train.csv', low_memory=False)
    store = pd.read_csv('/kaggle/input/rossmann-store-sales/store.csv', low_memory=False)
    test = pd.read_csv('/kaggle/input/rossmann-store-sales/test.csv', low_memory=False)
    sample = pd.read_csv('/kaggle/input/rossmann-store-sales/sample_submission.csv', low_memory=False)
    return data, test, store, sample

In [ ]:
data, test, store, sample = read_all_data()

In [ ]:
def calc_iqr(ps):
    q1 = ps.quantile(0.25)
    q3 = ps.quantile(0.75)
    iqr = q3 - q1
    return iqr

def get_customers_info():
    data = pd.read_csv('/kaggle/input/rossmann-store-sales/train.csv', low_memory=False)
    store_avg_cust = data.groupby(['Store']).agg(avg_cust=('Customers', 'mean'),
                                                 median_cust=('Customers', 'median'),
                                                 iqr_cust=('Customers', calc_iqr)).reset_index()
    
    store_avg_cust_dow = data.groupby(['Store', 'DayOfWeek']).agg(avg_cust_dow=('Customers', 'mean'),
                                                                  median_cust_dow=('Customers', 'median'),
                                                                  iqr_cust_dow=('Customers', calc_iqr)).reset_index()
    
    store_avg_cust_promo = data.groupby(['Store', 'Promo']).agg(avg_cust_promo=('Customers', 'mean'),
                                                                median_cust_promo=('Customers', 'median'),
                                                                iqr_cust_promo=('Customers', calc_iqr)).reset_index()
    return store_avg_cust, store_avg_cust_dow, store_avg_cust_promo



def fill_store_data(store):
    store['CompetitionDistance'] = store['CompetitionDistance'].fillna(store['CompetitionDistance'].median())
    store['PromoInterval'] = store['PromoInterval'].fillna(store['PromoInterval'].mode().iloc[0])
    store['CompetitionOpenSinceMonth'] = store['CompetitionOpenSinceMonth'].fillna(0)
    store['CompetitionOpenSinceYear'] = store['CompetitionOpenSinceYear'].fillna(store['CompetitionOpenSinceYear'].mode().iloc[0])
    store['Promo2SinceWeek'] = store['Promo2SinceWeek'].fillna(0)
    store['Promo2SinceYear'] = store['Promo2SinceYear'].fillna(store['Promo2SinceYear'].mode().iloc[0])
    return store

def feature_engg(df, store):
    df['is_december'] = df['month'].apply(lambda v: 1 if v==12 else 0)
    df['is_state_holiday_flg'] = df['StateHoliday'].apply(lambda v: 0 if v==0 else 1)
    df['is_holiday'] = df['is_state_holiday_flg'] * df['SchoolHoliday']
    # store_avg_cust = get_customers_info()
    # store_avg_cust, store_avg_cust_dow = get_customers_info()
    store_avg_cust, store_avg_cust_dow, store_avg_cust_promo = get_customers_info()
    df = pd.merge(left=df, right=store_avg_cust, on='Store', how='left')
    df = pd.merge(left=df, right=store_avg_cust_dow, on=['Store', 'DayOfWeek'], how='left')
    df = pd.merge(left=df, right=store_avg_cust_promo, on=['Store', 'Promo'], how='left')
    return df

def preprocess(df, store):
    store = fill_store_data(store)
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
    df['day'] = df.Date.dt.day
    df['month'] = df.Date.dt.month
    df['year'] = df.Date.dt.year
    df['StateHoliday'] = df['StateHoliday'].map({'0':0, 0:0, 'a':1, 'b':2, 'c':3})
    df = feature_engg(df, store)
    df = pd.merge(left=df, right=store, on='Store', how='left')
    return df

calc_rmse = lambda mse: np.sqrt(mse)



In [ ]:
data, test, store, sample = read_all_data()
data = preprocess(data, store)
data = data[data['Open'] != 0]
test = preprocess(test, store)

data_dummies = pd.get_dummies(data.drop(['Customers', 'Date'], axis=1), drop_first=True)
test_dummies = pd.get_dummies(test.drop(['Date', 'Id'], axis=1), drop_first=True)
data_dummies.shape, test_dummies.shape

target_col = 'Sales'
input_cols = data_dummies.columns.drop([target_col])
train_x, validate_x, train_y, validate_y = train_test_split(data_dummies[input_cols],
                                                            data_dummies[target_col],
                                                            test_size=0.2,
                                                            random_state=1)
train_x.shape, validate_x.shape, train_y.shape, validate_y.shape

In [ ]:
model = DecisionTreeRegressor(max_depth=12, random_state=1).fit(train_x, train_y)
validate_y_pred = model.predict(validate_x)
mse = mean_squared_error(validate_y, validate_y_pred)
rmse = calc_rmse(mse)
print(rmse)

In [ ]:
test_y_pred = model.predict(test_dummies.fillna(0))
test['Sales'] = test_y_pred
closed_dates_indexes = test[test['Open']==0].index
test.loc[closed_dates_indexes, 'Sales'] = 0
test[['Id', 'Sales']].to_csv('submission.csv', index=False)

In [ ]:
df_feature_imp = pd.DataFrame(model.feature_importances_, index=train_x.columns, columns=['score'])
df_feature_imp.sort_values('score', ascending=False).plot.bar(figsize=(20, 4))

## Hyperparameter Tuning

In [ ]:
'''
base_model = DecisionTreeRegressor()
param_grid = {'max_depth': [7, 9, 11, 13, 15]}
tuner = GridSearchCV(estimator=base_model, cv=5, param_grid=param_grid, return_train_score=True).fit(train_x, train_y)
'''

In [ ]:
#df_cv = pd.DataFrame(tuner.cv_results_)
#df_cv[['param_max_depth', 'mean_test_score', 'rank_test_score']].sort_values('mean_test_score', ascending=False)

In [ ]:
#import matplotlib.pyplot as plt
#sns.lineplot(data=df_cv, x='param_max_depth', y='mean_train_score')
#sns.lineplot(data=df_cv, x='param_max_depth', y='mean_test_score')
#plt.legend(['Training score', 'Testing Score'])